In [1]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import psycopg2

### Source: Kaggle 
* https://www.kaggle.com/tobycrabtree/nfl-scores-and-betting-data
* downloaded teams, scores, and stadiums data from Kaggle

In [2]:
# Files to Load 
team_df = pd.read_csv('teams.csv')
stadium_data = pd.read_csv('stadiums.csv')
score_data = pd.read_csv('scores.csv')

### NFL Teams 
* The National Football League (NFL) is a professional American football league consisting of 32 teams, divided equally between the National Football Conference (NFC) and the American Football Conference (AFC).

In [3]:
team_df.drop(['team_id_pfr'],axis='columns',inplace=True)

In [4]:
team_df = team_df.replace(np.nan, '', regex=True)
team_df.head()

,team_name,team_name_short,team_id,team_conference,team_division,team_conference_pre2002,team_division_pre2002
0,Arizona Cardinals,Cardinals,ARI,NFC,NFC West,NFC,NFC West
1,Phoenix Cardinals,Cardinals,ARI,NFC,,NFC,NFC East
2,St. Louis Cardinals,Cardinals,ARI,NFC,,NFC,NFC East
3,Atlanta Falcons,Falcons,ATL,NFC,NFC South,NFC,NFC West
4,Baltimore Ravens,Ravens,BAL,AFC,AFC North,AFC,AFC Central


In [5]:
team_df.set_index("team_name", inplace=True)
team_df.head()

,team_name_short,team_id,team_conference,team_division,team_conference_pre2002,team_division_pre2002
team_name,,,,,,
Arizona Cardinals,Cardinals,ARI,NFC,NFC West,NFC,NFC West
Phoenix Cardinals,Cardinals,ARI,NFC,,NFC,NFC East
St. Louis Cardinals,Cardinals,ARI,NFC,,NFC,NFC East
Atlanta Falcons,Falcons,ATL,NFC,NFC South,NFC,NFC West
Baltimore Ravens,Ravens,BAL,AFC,AFC North,AFC,AFC Central


In [6]:
team_df.dtypes

team_name_short            object
team_id                    object
team_conference            object
team_division              object
team_conference_pre2002    object
team_division_pre2002      object
dtype: object

### List of current National Football League stadiums
* Stadiums represent a considerable expense to a community, and thus their construction, use, and funding often enter the public discourse.[1] Also, given the perceived advantage a team gets to playing in its home stadium, particular attention is given in the media to the peculiarities of each stadium's environment. Climate, playing surface (either natural or artificial turf), and the type of roof all contribute to giving each team its home-field advantage.

* Stadiums are either open, enclosed, or have a retractable roof. For retractable roofs, the home team determines if the roof is to be opened or closed 90 minutes before kickoff. The roof remains open unless precipitation or lightning is within the vicinity of the stadium, the temperature drops below 40 °F (4 °C), or wind gusts are greater than 40 miles per hour (64 km/h), in which case the roof operators will close the roof.[2]

In [7]:
# create a list of columns that you are considering to keep
stadium_cols = ['stadium_name', 'stadium_location', 'stadium_type', 'stadium_weather_type', 'stadium_capacity']

# create a new dataframe from the original
stadium_df = stadium_data[stadium_cols].copy()
stadium_df.head()

,stadium_name,stadium_location,stadium_type,stadium_weather_type,stadium_capacity
0,Alamo Dome,"San Antonio, TX",indoor,dome,72000
1,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN
2,Alumni Stadium,"Chestnut Hill, MA",outdoor,cold,NaN
3,Anaheim Stadium,"Anaheim, CA",outdoor,warm,NaN
4,Arrowhead Stadium,"Kansas City, MO",outdoor,cold,"76,416"


In [8]:
# extract city and state from stadium location and add them as separate columns
def get_city(stadium_location):
    return stadium_location.split(",")[0]

def get_state(stadium_location):
    return stadium_location.split(",")[1]

stadium_df['city'] = stadium_df['stadium_location'].apply(lambda x: f"{get_city(x)}")
stadium_df['state'] = stadium_df['stadium_location'].apply(lambda x: f"{get_state(x)}")
stadium_df.head()

,stadium_name,stadium_location,stadium_type,stadium_weather_type,stadium_capacity,city,state
0,Alamo Dome,"San Antonio, TX",indoor,dome,72000,San Antonio,TX
1,Alltel Stadium,"Jacksonville, FL",NaN,NaN,NaN,Jacksonville,FL
2,Alumni Stadium,"Chestnut Hill, MA",outdoor,cold,NaN,Chestnut Hill,MA
3,Anaheim Stadium,"Anaheim, CA",outdoor,warm,NaN,Anaheim,CA
4,Arrowhead Stadium,"Kansas City, MO",outdoor,cold,"76,416",Kansas City,MO


In [9]:
# drop stadium_laction
stadium_df.drop(['stadium_location'],axis='columns',inplace=True)
stadium_df.head()

,stadium_name,stadium_type,stadium_weather_type,stadium_capacity,city,state
0,Alamo Dome,indoor,dome,72000,San Antonio,TX
1,Alltel Stadium,NaN,NaN,NaN,Jacksonville,FL
2,Alumni Stadium,outdoor,cold,NaN,Chestnut Hill,MA
3,Anaheim Stadium,outdoor,warm,NaN,Anaheim,CA
4,Arrowhead Stadium,outdoor,cold,"76,416",Kansas City,MO


In [10]:
# set index to stadium_name
stadium_df.set_index("stadium_name", inplace=True)
stadium_df.tail()

,stadium_type,stadium_weather_type,stadium_capacity,city,state
stadium_name,,,,,
War Memorial Stadium,outdoor,cold,NaN,Buffalo,NY
Wembley Stadium,outdoor,moderate,NaN,London,UK
Wrigley Field,outdoor,cold,NaN,Chicago,IL
Yale Bowl,outdoor,cold,NaN,New Haven,CT
Yankee Stadium,outdoor,cold,NaN,Bronx,NY


In [11]:
stadium_df.head()

,stadium_type,stadium_weather_type,stadium_capacity,city,state
stadium_name,,,,,
Alamo Dome,indoor,dome,72000,San Antonio,TX
Alltel Stadium,NaN,NaN,NaN,Jacksonville,FL
Alumni Stadium,outdoor,cold,NaN,Chestnut Hill,MA
Anaheim Stadium,outdoor,warm,NaN,Anaheim,CA
Arrowhead Stadium,outdoor,cold,"76,416",Kansas City,MO


### Scores

* Scores for all the teams since 1966

In [12]:
# create a list of columns that you are considering to keep
score_cols = ['schedule_season', 'schedule_week', 'schedule_playoff', 'team_home', 'score_home', 
              'score_away', 'team_away',  'stadium']

# create a new dataframe from original
score_df = score_data[score_cols].copy()

score_df

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,stadium
0,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,Orange Bowl
1,1966,1,False,Houston Oilers,45,7,Denver Broncos,Rice Stadium
2,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,Balboa Stadium
3,1966,2,False,Miami Dolphins,14,19,New York Jets,Orange Bowl
4,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,Lambeau Field
...,...,...,...,...,...,...,...,...
12673,2019,Division,True,Green Bay Packers,28,23,Seattle Seahawks,Lambeau Field
12674,2019,Division,True,Kansas City Chiefs,51,31,Houston Texans,Arrowhead Stadium
12675,2019,Conference,True,Kansas City Chiefs,35,24,Tennessee Titans,Arrowhead Stadium
12676,2019,Conference,True,San Francisco 49ers,37,20,Green Bay Packers,Levi's Stadium


In [13]:
# add the winner column based on scores between home team and opponent
score_df['winner'] = np.where(score_df['score_home'] > score_df['score_away'], score_df['team_home'], score_df['team_away']) 
score_df.head()

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,stadium,winner
0,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,Orange Bowl,Oakland Raiders
1,1966,1,False,Houston Oilers,45,7,Denver Broncos,Rice Stadium,Houston Oilers
2,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,Balboa Stadium,San Diego Chargers
3,1966,2,False,Miami Dolphins,14,19,New York Jets,Orange Bowl,New York Jets
4,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,Lambeau Field,Green Bay Packers


In [14]:
# add id column 
score_df['id'] = range(1, 1+len(score_df))
score_df.head()

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,stadium,winner,id
0,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,Orange Bowl,Oakland Raiders,1
1,1966,1,False,Houston Oilers,45,7,Denver Broncos,Rice Stadium,Houston Oilers,2
2,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,Balboa Stadium,San Diego Chargers,3
3,1966,2,False,Miami Dolphins,14,19,New York Jets,Orange Bowl,New York Jets,4
4,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,Lambeau Field,Green Bay Packers,5


In [15]:
# move the id column to first place
cols = score_df.columns.tolist()
cols = cols[-1:] + cols[:-1]
score_df = score_df[cols] 
score_df.head()

,id,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,stadium,winner
0,1,1966,1,False,Miami Dolphins,14,23,Oakland Raiders,Orange Bowl,Oakland Raiders
1,2,1966,1,False,Houston Oilers,45,7,Denver Broncos,Rice Stadium,Houston Oilers
2,3,1966,1,False,San Diego Chargers,27,7,Buffalo Bills,Balboa Stadium,San Diego Chargers
3,4,1966,2,False,Miami Dolphins,14,19,New York Jets,Orange Bowl,New York Jets
4,5,1966,1,False,Green Bay Packers,24,3,Baltimore Colts,Lambeau Field,Green Bay Packers


In [16]:
score_df.set_index("id", inplace=True)
score_df.tail()

,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,stadium,winner
id,,,,,,,,,
12674,2019,Division,True,Green Bay Packers,28,23,Seattle Seahawks,Lambeau Field,Green Bay Packers
12675,2019,Division,True,Kansas City Chiefs,51,31,Houston Texans,Arrowhead Stadium,Kansas City Chiefs
12676,2019,Conference,True,Kansas City Chiefs,35,24,Tennessee Titans,Arrowhead Stadium,Kansas City Chiefs
12677,2019,Conference,True,San Francisco 49ers,37,20,Green Bay Packers,Levi's Stadium,San Francisco 49ers
12678,2019,Superbowl,True,Kansas City Chiefs,31,20,San Francisco 49ers,Hard Rock Stadium,Kansas City Chiefs


### Create database connection

In [17]:
connection_string = "postgres:postgres@localhost:5432/nfl_db"
engine = create_engine(f'postgresql://{connection_string}')

### Load DataFrames into database

In [18]:
team_df.to_sql(name='teams', con=engine, if_exists='replace', index=True)

In [19]:
stadium_df.to_sql(name='stadiums', con=engine, if_exists='replace', index=True)

In [20]:
score_df.to_sql(name='scores', con=engine, if_exists='replace', index=True)